In [1]:
import requests
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:
env_path = os.path.join('misc', '.env')
load_dotenv(dotenv_path=env_path)
SPOTIFY_CREDS = [os.getenv('SPOTIFY_CLIENT_ID'), 
                 os.getenv('SPOTIFY_CLIENT_SECRET')]

if not all(SPOTIFY_CREDS):
    print("Please set the SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET environment variables")
    exit(1)

In [3]:
def get_access_token(creds: list[str]) -> str:
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": creds[0],
        "client_secret": creds[1]
    }
    access_token = None
    try:
        res = requests.post(url, headers=headers, data=data)
    except requests.exceptions.RequestException as e:
        print("Failed to get token: ", e)
        exit(1)
    else:
        if res.status_code < 400:
            access_token = res.json()['access_token']
    return access_token

In [4]:
access_token = get_access_token(SPOTIFY_CREDS)
if not access_token:
    print("Failed to get access token")
    exit(1)

track_data = pd.read_csv('misc/music_info.csv')
track_data = track_data.drop(['spotify_preview_url'], axis=1)
track_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          50683 non-null  object 
 1   name              50683 non-null  object 
 2   artist            50683 non-null  object 
 3   spotify_id        50683 non-null  object 
 4   tags              49556 non-null  object 
 5   genre             22348 non-null  object 
 6   year              50683 non-null  int64  
 7   duration_ms       50683 non-null  int64  
 8   danceability      50683 non-null  float64
 9   energy            50683 non-null  float64
 10  key               50683 non-null  int64  
 11  loudness          50683 non-null  float64
 12  mode              50683 non-null  int64  
 13  speechiness       50683 non-null  float64
 14  acousticness      50683 non-null  float64
 15  instrumentalness  50683 non-null  float64
 16  liveness          50683 non-null  float6

In [5]:
api_endpoint = "https://api.spotify.com/v1"
def get_recommendations(seed_tracks: list[str], access_token: str):
    url = f"{api_endpoint}/recommendations"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "seed_tracks": ",".join(seed_tracks),
        "limit": 5
    }
    try:
        res = requests.get(url, headers=headers, params=params)
    except requests.exceptions.RequestException as e:
        print("Failed to get recommendations: ", e)
        exit(1)
    else:
        if res.status_code < 400:
            return res.json()
        
def get_track_info(track_id: str, access_token: str):
    url = f"{api_endpoint}/audio-features/{track_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    try:
        res = requests.get(url, headers=headers)
    except requests.exceptions.RequestException as e:
        print("Failed to get track info: ", e)
        exit(1)
    else:
        if res.status_code < 400:
            return res.json()

In [6]:
random_tracks = track_data.sample(5)
seed_tracks = random_tracks['spotify_id'].tolist()

random_tracks

,track_id,name,artist,spotify_id,tags,genre,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
47793,TRVOAQZ128F931A288,Stromausfall im Herzspital,Samsas Traum,0sHc5jIomO3YVOp1XS45d9,gothic,NaN,2013,269773,0.725,0.772,0,-4.994,0,0.0309,0.123000,0.011500,0.1850,0.420,110.006,4
31678,TROQCJZ128F9337DEA,Good to Be Alive,Matthew Dear,0pO9x8UEWoABfAZY0P1otB,"electronic, techno, idm",Electronic,2008,228906,0.921,0.894,0,-7.354,1,0.1490,0.259000,0.742000,0.1250,0.816,106.019,4
28857,TRCXOTB12903CDB1F4,Drifting Apart,Alien Ant Farm,2U3UdAv17dF06HSLrH81lT,"rock, alternative_rock, trance",NaN,2003,174242,0.459,0.951,5,-2.571,1,0.0486,0.000074,0.000000,0.0764,0.503,96.479,4
10616,TRCQUHU128F426CA10,Pump It Up,Elvis Costello & The Attractions,0JgMHia55MBfhfqZIRi3kF,"rock, classic_rock, punk, singer_songwriter, 8...",NaN,1978,196680,0.644,0.814,11,-6.036,1,0.0385,0.009330,0.000617,0.1070,0.965,138.999,4
23197,TRYBWEB128E0791898,Globes & Maps,Something Corporate,2eJ7yBYWPFgDfcNaPJN0It,"soundtrack, piano, emo",NaN,2002,288040,0.494,0.303,0,-8.221,1,0.0309,0.805000,0.000000,0.1830,0.239,131.195,4


In [7]:
recommendations = get_recommendations(seed_tracks, access_token)

In [8]:
predicted_songs = []
for track in recommendations['tracks']:
    track_id = track['external_urls']['spotify'].split('/')[-1]
    track_features = get_track_info(track_id, access_token)
    track = {
        "id": track_id,
        "name": track['name'],
        "artist": track['artists'][0]['name'],
    }
    track.update(track_features)
    predicted_songs.append(track)

predicted_songs = pd.DataFrame(predicted_songs)
predicted_songs
    


,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,0PW3VQvrc6u42hK5VpGtWE,Reichtum,Coppelius,0.717,0.664,0,-5.969,1,0.0424,0.002900,0.027600,0.0702,0.634,105.036,audio_features,spotify:track:0PW3VQvrc6u42hK5VpGtWE,https://api.spotify.com/v1/tracks/0PW3VQvrc6u4...,https://api.spotify.com/v1/audio-analysis/0PW3...,197173,4
1,0OZqRdXHAwxPXdg7W0Chxt,To the Last Beat of My Heart,Fujiya & Miyagi,0.651,0.788,8,-9.195,1,0.0371,0.085700,0.889000,0.1070,0.285,107.997,audio_features,spotify:track:0OZqRdXHAwxPXdg7W0Chxt,https://api.spotify.com/v1/tracks/0OZqRdXHAwxP...,https://api.spotify.com/v1/audio-analysis/0OZq...,227695,4
2,2R6UrJ8uWbSIiHWmvRQvN8,Whiskey In The Jar,Metallica,0.511,0.970,10,-3.720,1,0.0414,0.000006,0.001360,0.1800,0.566,132.986,audio_features,spotify:track:2R6UrJ8uWbSIiHWmvRQvN8,https://api.spotify.com/v1/tracks/2R6UrJ8uWbSI...,https://api.spotify.com/v1/audio-analysis/2R6U...,304693,4
3,0INUE1K7cEiE8VH63Rv5RJ,Happy?,Mudvayne,0.546,0.970,5,-2.661,0,0.0970,0.000823,0.000021,0.2750,0.383,100.065,audio_features,spotify:track:0INUE1K7cEiE8VH63Rv5RJ,https://api.spotify.com/v1/tracks/0INUE1K7cEiE...,https://api.spotify.com/v1/audio-analysis/0INU...,216427,4
4,3hOFPVboSZkpBhsUsaWYdD,The Birds - Matthew Dear Remix,Telefon Tel Aviv,0.634,0.846,11,-7.447,0,0.0418,0.052100,0.831000,0.1020,0.125,109.962,audio_features,spotify:track:3hOFPVboSZkpBhsUsaWYdD,https://api.spotify.com/v1/tracks/3hOFPVboSZkp...,https://api.spotify.com/v1/audio-analysis/3hOF...,327192,4
